### Подход 1: градиентный бустинг "в лоб"

In [1]:
import numpy as np
import pandas as p
from pandas import DataFrame

features = p.read_csv('./features.csv', index_col='match_id')
features_train = features.drop(columns=[
     "duration", 
     "radiant_win",
     "tower_status_radiant",
     "tower_status_dire",
     "barracks_status_dire",
     "barracks_status_radiant"
])

In [2]:
p.options.display.max_rows = len(features_train)
values_count = features_train.count().sort_values(ascending=True)

#### Какие признаки имеют пропуски среди своих значений? Что могут означать пропуски в этих признаках (ответьте на этот вопрос для двух любых признаков)?

In [26]:
rows = values_count.max()

print("Количество пропусков\n")
for (feature, val) in sorted(values_count.iteritems()):
    values_missing = rows - val
    if values_missing > 0:
        print(f"{feature}: {values_missing}")

Количество пропусков

dire_bottle_time: 16143
dire_courier_time: 676
dire_first_ward_time: 1826
dire_flying_courier_time: 26098
first_blood_player1: 19553
first_blood_player2: 43987
first_blood_team: 19553
first_blood_time: 19553
radiant_bottle_time: 15691
radiant_courier_time: 692
radiant_first_ward_time: 1836
radiant_flying_courier_time: 27479


Много значений пропущено в колонках, касающихся First Blood (first_blood_player1, first_blood_player2, first_blood_time, first_blood_team). Это значит, что игроки никого не убили за первые 5 минут матча.

Пропуски в *_bottle_time, *_courier_time и *_flying_courier_time означают, что соответствующая команда не купила эти предметы за первые 5 минут.

Пропуски в *_first_ward_time означают, что команда не поставила ни одного наблюдателя за 5 минут.

In [28]:
features_train = features_train.fillna(value=0)

#### 2. Как называется столбец, содержащий целевую переменную?

_radiant_win_

#### 3. Как долго проводилась кросс-валидация для градиентного бустинга с 30 деревьями? Инструкцию по измерению времени можно найти ниже по тексту. Какое качество при этом получилось? Напомним, что в данном задании мы используем метрику качества AUC-ROC.